In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# !pip install tqdm pandas wandb librosa numpy torchmetrics torch
# !pip3 install tqdm

In [3]:
import warnings
from tqdm import tqdm
import os
from pathlib import Path
import math
import pandas as pd

import wandb

import librosa

import numpy as np

import torchmetrics
import torchmetrics.classification

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import LambdaLR

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preparing the dataset

This notebook will use the global audio datasets for the emotion recognition model. This will give us the relative success of the model compared to pre-existing models

## Create csv file saving the location of the global datset.

We are using the TESS, RAV, SAVEE and CREMA datasets

In [4]:
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

TESS = "/kaggle/input/toronto-emotional-speech-set-tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data/"
RAV = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
# SAVEE = "/kaggle/input/surrey-audiovisual-expressed-emotion-savee/ALL/"
# CREMA = "/kaggle/input/cremad/AudioWAV/"


# Run one example 
dir_list = os.listdir(RAV)
dir_list[0:5]

['Actor_02', 'Actor_17', 'Actor_05', 'Actor_16', 'Actor_21']

RAVDESS editiong

In [5]:
dir_list = os.listdir(RAV)
dir_list.sort()

emotion = []
gender = []
path = []
for i in dir_list:
    fname = os.listdir(RAV + i)
    for f in fname:
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        temp = int(part[6])
        if temp%2 == 0:
            temp = "female"
        else:
            temp = "male"
        gender.append(temp)
        path.append(RAV + i + '/' + f)

        
RAV_df = pd.DataFrame(emotion)
RAV_df = RAV_df.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
RAV_df = pd.concat([pd.DataFrame(gender),RAV_df],axis=1)
RAV_df.columns = ['gender','emotion']
RAV_df['labels'] =RAV_df.gender + '_' + RAV_df.emotion
RAV_df['source'] = 'RAVDESS'  
RAV_df = pd.concat([RAV_df,pd.DataFrame(path, columns = ['path'])],axis=1)
RAV_df = RAV_df.drop(['gender', 'emotion'], axis=1)
RAV_df.labels.value_counts()

labels
male_neutral       144
female_neutral     144
male_fear           96
male_happy          96
male_disgust        96
male_sad            96
male_angry          96
male_surprise       96
female_surprise     96
female_disgust      96
female_fear         96
female_sad          96
female_happy        96
female_angry        96
Name: count, dtype: int64

In [6]:
dir_list = os.listdir(TESS)
dir_list.sort()

path = []
emotion = []

for i in dir_list:
    fname = os.listdir(TESS + i)
    for f in fname:
        if i == 'OAF_angry' or i == 'YAF_angry':
            emotion.append('female_angry')
        elif i == 'OAF_disgust' or i == 'YAF_disgust':
            emotion.append('female_disgust')
        elif i == 'OAF_Fear' or i == 'YAF_fear':
            emotion.append('female_fear')
        elif i == 'OAF_happy' or i == 'YAF_happy':
            emotion.append('female_happy')
        elif i == 'OAF_neutral' or i == 'YAF_neutral':
            emotion.append('female_neutral')                                
        elif i == 'OAF_Pleasant_surprise' or i == 'YAF_pleasant_surprised':
            emotion.append('female_surprise')               
        elif i == 'OAF_Sad' or i == 'YAF_sad':
            emotion.append('female_sad')
        else:
            emotion.append('Unknown')
        path.append(TESS + i + "/" + f)

TESS_df = pd.DataFrame(emotion, columns = ['labels'])
TESS_df['source'] = 'TESS'
TESS_df = pd.concat([TESS_df,pd.DataFrame(path, columns = ['path'])],axis=1)
TESS_df.labels.value_counts()

labels
female_fear        400
female_surprise    400
female_sad         400
female_angry       400
female_disgust     400
female_happy       400
female_neutral     400
Name: count, dtype: int64

In [7]:
df = pd.concat([RAV_df, TESS_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)

labels
female_neutral     544
female_sad         496
female_fear        496
female_disgust     496
female_surprise    496
female_happy       496
female_angry       496
male_neutral       144
male_angry          96
male_fear           96
male_sad            96
male_disgust        96
male_happy          96
male_surprise       96
Name: count, dtype: int64


In [8]:
ref = pd.read_csv("/kaggle/working/Data_path.csv")

In [9]:
class AudioDataset(Dataset):
    def __init__(self, data, seq_len, d_model, augment=False):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model
        self.data = data
        self.augment = augment
        self.num_augments = 1  # Default number of augmentations (original only)

        if self.augment:
            self.num_augments = 4   # Add augment parameter

        self.class_map = {
            "female_neutral": 0, "female_surprise": 1, "female_disgust": 2, 
            "female_fear": 3, "female_sad": 4, "female_happy": 5, "female_angry": 6, 
            "male_neutral": 7, "male_sad": 8, "male_fear": 9, "male_happy": 10, 
            "male_disgust": 11, "male_angry": 12, "male_surprise": 13
        }
        self.label_map = {0: "female_neutral", 1: "female_surprise", 2: "female_disgust", 
                          3: "female_fear", 4: "female_sad", 5: "female_happy", 6: "female_angry", 
                          7: "male_neutral", 8: "male_sad", 9: "male_fear", 10: "male_happy",
                          11: "male_disgust", 12: "male_angry", 13: "male_surprise"}

    @staticmethod
    def extract_audio_features(signal, sample_rate=44100):
        # MFCC
        mfccs = librosa.feature.mfcc(y=signal, n_mfcc=13, sr=sample_rate)
        delta_mfccs = librosa.feature.delta(mfccs)
        delta2_mfccs = librosa.feature.delta(mfccs, order=2)
        # Root Mean Square Energy
#         root_mean_out = librosa.feature.rms(y=signal)
        mfccs_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
        return mfccs_features

    def __len__(self):
        return len(self.data) * self.num_augments

    
    def __getitem__(self, idx):
        # Determine which original sample and which augmentation to use
        original_idx = idx // self.num_augments
        augment_idx = idx % self.num_augments

        file_path, emotion = self.data[original_idx]
        signal, sr = librosa.load(file_path)

        # Create a list of signals (including augmentations)
        signals = [signal]  # Start with the original signal

        if self.augment:
            noisy_signal = self.add_noise(signal)
            stretch_signal = self.stretch_process(signal)
            pitch_signal = self.pitch_process(signal, sr)
            signals.extend([noisy_signal, stretch_signal, pitch_signal])

        # Select the signal based on the augmentation index
        sig = signals[augment_idx]
        inp = AudioDataset.extract_audio_features(sig, sr)

        # Padding
        padding = self.seq_len - inp.shape[1]
        if padding < 0:
            raise ValueError("Audio is too long")
        
        inp = torch.cat(
            [
                torch.tensor(inp, dtype=torch.float32),
                torch.zeros((self.d_model, padding), dtype=torch.float32)
            ], 1
        )

        label = torch.zeros(14)
        label[self.class_map[emotion]] = 1

        sample = {
            "label": label,
            "file_path": file_path,
            "input": inp.T,
            "emotion": emotion,
            "class": self.class_map[emotion]
        }
        
        return sample

    
    @staticmethod
    def add_noise(signal, noise_level=0.005):
        noise = np.random.randn(len(signal))
        signal_noisy = signal + noise_level * noise
        return signal_noisy

    @staticmethod
    def stretch_process(signal, rate=1.2):
        return librosa.effects.time_stretch(signal, rate=rate)

    @staticmethod
    def pitch_process(signal, sr, n_steps=4):
        return librosa.effects.pitch_shift(signal, sr=sr, n_steps=n_steps)

    
    
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0


def get_label(vector):
    # class_map = {"anger" : 0, "sadness": 1, "fear": 2, "happy": 3, "neutral": 4, "surprise": 5, "sarcastic": 6, "disgust": 7}
    label_map = {0: "anger", 1: "sadness", 2: "fear", 3: "happy", 4: "neutral", 5: "surprise", 6: "sarcastic", 7: "disgust"}

    return label_map[np.argmax(vector)]

In [10]:
class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # eps is to prevent dividing by zero or when std is very small
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        self.dropout_4 = nn.Dropout(dropout)
        
        # linear model
        self.linear_1 = nn.Linear(d_model, d_ff) 
        self.linear_2 = nn.Linear(d_ff, d_ff)
        self.linear_3 = nn.Linear(d_ff, d_ff)
        self.linear_4 = nn.Linear(d_ff, d_ff)        
        self.linear_5 = nn.Linear(d_ff, d_model) 
        
        # cnn model
        # self.conv_1 = nn.Conv2d(1, 8, kernel_size=(2, 2), padding=1, stride=1)
        # self.conv_2 = nn.Conv2d(8, 64, kernel_size=(2, 2), padding=1, stride=1)
        # self.conv_3 = nn.Conv2d(64, 128, kernel_size=(2, 2), padding=1, stride=1)
        # self.conv_4 = nn.Conv2d(128, 64, kernel_size=(2, 2), padding=1, stride=1)
        # self.conv_5 = nn.Conv2d(64, 8, kernel_size=(2, 2), padding=1, stride=1)
        # self.conv_6 = nn.Conv2d(8, 1, kernel_size=(2, 2), padding=1, stride=1)

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        x = self.linear_1(x)
        x = self.linear_2(self.dropout_1(torch.relu(x)))
        x = self.linear_3(self.dropout_2(torch.relu(x)))
        x = self.linear_4(self.dropout_3(torch.relu(x)))
        x = self.linear_5(self.dropout_4(torch.relu(x)))
        
        # convo forward 
        # (batch, seq_len, d_model) --> (batch, 1, seq_len, d_model)
        # x = x.unsqeeze(1)
        
        return x

    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        sin_div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        cos_div_term = torch.exp(torch.arange(1, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * sin_div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * cos_div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

class ResidualConnection(nn.Module):
    
        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)
    
        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask = None , dropout: nn.Dropout = None):
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask = None):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)  
        return self.w_o(x)

class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask = None):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x
    
    
class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask = None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


class ProjectionLayer(nn.Module):

    def __init__(self, seq_len: int, d_model: int, d_ff: int, num_of_labels: int, dropout: float) -> None:
        super().__init__()
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        self.dropout_4 = nn.Dropout(dropout)
        self.dropout_5 = nn.Dropout(dropout)
        self.dropout_6 = nn.Dropout(dropout)
        
        self.proj = nn.Linear(d_ff, num_of_labels)

#         self.proj = nn.Linear(512, num_of_labels)
        # dense network
#         self.linear_1 = nn.Linear(d_model, d_ff)
#         self.linear_2 = nn.Linear(d_ff, d_ff)
#         self.linear_3 = nn.Linear(d_ff, d_model)
#         self.linear_4 = nn.Linear(d_model*seq_len, d_ff)
#         self.linear_5 = nn.Linear(d_ff, d_ff)
#         self.linear_6 = nn.Linear(d_ff, d_ff)
        
        
        # cnn model
        # self.conv_1 = nn.Conv2d(1, 8, padding_mode='replicate')
        # self.conv_2 = nn.Conv2d(8, 64, padding_mode='replicate')
        # self.conv_3 = nn.Conv2d(64, 8, padding_mode='replicate')
        # self.conv_4 = nn.Conv2d(8, 1, padding_mode='replicate')
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.batchnorm_1 = nn.BatchNorm2d(8)
        self.pool_1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batchnorm_2 = nn.BatchNorm2d(64)
        self.pool_2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.batchnorm_3 = nn.BatchNorm2d(128)
        self.pool_3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=d_ff, kernel_size=(75, 4), stride=1, padding=0)
        self.batchnorm_4 = nn.BatchNorm2d(d_ff)


    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, num_of_labels)
#         x = self.linear_1(x)
#         x = self.linear_2(self.dropout_1(torch.relu(x)))
#         x = self.linear_3(self.dropout_2(torch.relu(x)))
#         x = torch.flatten(x,1)
#         x = self.linear_4(self.dropout_3(torch.relu(x)))
#         x = self.linear_5(self.dropout_4(torch.relu(x)))
#         x = self.linear_6(self.dropout_5(torch.relu(x)))
#         x = self.proj(self.dropout_6(torch.relu(x)))
        
        # convo forward 
        # (batch, seq_len, d_model) --> (batch, 1, seq_len, d_model)
        x = torch.unsqueeze(x, 1)
        x = self.dropout_1(self.pool_1(torch.relu(self.batchnorm_1(self.conv1(x)))))
        x = self.dropout_2(self.pool_2(torch.relu(self.batchnorm_2(self.conv2(x)))))
        x = self.dropout_3(self.pool_3(torch.relu(self.batchnorm_3(self.conv3(x)))))
        x = self.dropout_4(torch.relu(self.batchnorm_4(self.conv4(x))))
        
        x = torch.flatten(x, 1)
        x = self.proj(x)
        
        return x



    
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, inp_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.inp_pos = inp_pos
        self.projection_layer = projection_layer

    def encode(self, inp, inp_mask = None):
        # (batch, seq_len, d_model)
        inp = self.inp_pos(inp)
        return self.encoder(inp, inp_mask)
    
    def project(self, x):
        # (batch, num_of_label)
        return self.projection_layer(x)
    


def build_transformer(seq_len: int, num_of_labels: int, d_model: int=39, N: int=5, h: int=3, dropout: float=0.1, d_ff: int=256) -> Transformer:
    # Create the embedding layers

    # Create the positional encoding layers
    inp_pos = PositionalEncoding(d_model, seq_len, dropout)
    
    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    
    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    
    # Create the projection layer
    projection_layer = ProjectionLayer(seq_len, d_model, d_ff, num_of_labels, dropout)
    
    # Create the transformer
    transformer = Transformer(encoder, inp_pos, projection_layer)
    transformer.to(device)
    
    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return transformer


class SquareRootScheduler:
    def __init__(self, lr=0.1):
        self.lr = lr

    def __call__(self, num_update):
        return self.lr * pow(num_update + 1.0, -0.5)

In [11]:
def get_config():
    return {
        "dataset_root_location": "/kaggle/working/",
        "df_location": "Data_path.csv",
        "num_of_labels": 14,
        "batch_size": 16,
        "num_epochs": 50,
        "lr": 0.1,
        "seq_len": 600,
        "d_model": 39,
        "datasource": "Global_dataset_ravdess_tess",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": "latest",
        "experiment_name": "runs/tmodel"
    }

def get_weights_file_path(config, epoch: str):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)

# Find the latest weights file in the weights folder
def latest_weights_file_path(config):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}*"
    weights_files = list(Path(model_folder).glob(model_filename))
    if len(weights_files) == 0:
        return None
    weights_files.sort()
    return str(weights_files[-1])


In [12]:
# def run_validation(model, validation_ds, config, device, global_step):
#     model.eval()
#     count = 0

#     true_classes = []
#     pred_classes = []

#     # try:
#     #     # get the console window width
#     #     with os.popen('stty size', 'r') as console:
#     #         _, console_width = console.read().split()
#     #         console_width = int(console_width)
#     # except:
#     #     # If we can't get the console width, use 80 as default
#     #     console_width = 80

#     with torch.no_grad():
#         for batch in validation_ds:
#             count += 1
#             inp = batch["input"].to(device) # (b, d_model, seq_len)
#             inp_mask = None

#             # check that the batch size is 1
#             assert inp.size(
#                 0) == 1, "Batch size must be 1 for validation"

#             out = model.encode(inp)
#             out = model.project(out).squeeze(0)
#             # pred_class = (np.zeros_like(out) == 0)[np.argmax(out)] = 1

#             # storing results
#             pred_classes.append(torch.argmax(out).item())
#             true_classes.append(torch.argmax(batch["label"].squeeze(0)).item())
#         true_classes = torch.tensor(true_classes)
#         pred_classes = torch.tensor(pred_classes)
#         # Print the source, target and model output
#         # print_msg('-'*console_width)
#         # print_msg(f"{f'TARGET: ':>12}{label}")3
#         # print_msg(f"{f'PREDICTED: ':>12}{pred_out}")
#         # print_msg('-'*console_width)
    
    
#     # Evaluate the character error rate
#     # Compute the char error rate 
#     # metric = torchmetrics.classification.MulticlassConfusionMatrix(num_classes=config["num_of_labels"])
#     # cer = metric(pred_classes, true_classes)
#     # wandb.log({'validation/cer': cer, 'global_step': global_step})

#     # Compute confusion matrix
#     confusion_matrix = torchmetrics.ConfusionMatrix(num_classes=config["num_of_labels"], task="multiclass")
#     confusion_matrix(pred_classes, true_classes)

#     # Compute F1-score
#     f1_score = torchmetrics.classification.MulticlassF1Score(num_classes=config["num_of_labels"])
#     f1_score(pred_classes, true_classes)

#     # Compute accuracy
#     accuracy = torchmetrics.classification.MulticlassAccuracy(num_classes=config["num_of_labels"])
#     accuracy(pred_classes, true_classes) 

#     # Compute recall
#     recall = torchmetrics.classification.MulticlassRecall(num_classes=config["num_of_labels"])
#     recall(pred_classes, true_classes)   

#     # Compute precision
#     precision = torchmetrics.classification.MulticlassPrecision(num_classes=config["num_of_labels"])
#     precision(pred_classes, true_classes)

#     # Log metrics to WandB
#     wandb.log({"validation/confusion_matrix": confusion_matrix.compute(),
#                "validation/accuracy": accuracy.compute(), 
#                "validation/recall": recall.compute(), 
#                "validation/precision": precision.compute(), 
#                "validation/f1_score": f1_score.compute(),
#                'global_step': global_step})

# def get_models(config,num_models=3):
#     models=[]
#     for _ in range(numsm_models):
#         model = build_transformer(config["seq_len"], config['num_of_labels'], d_model=config['d_model'])
#         models.append(model)
        
#     return models

# def train_model(config, num_models=3):
#     # Define the device
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print("Using device:", device)

#     # Make sure the weights folder exists
#     Path(config['model_folder']).mkdir(parents=True, exist_ok=True)

#     train_dataloader, val_dataloader = get_ds(config)
# #     model = get_model(config).to(device)
#     models = get_models(config,num_models)
#     models = [model.to(device) for model in models]
    
# #     optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'])
#     optimizers = [torch.optim.SGD(model.parameters(), lr=config['lr']) for model in models]
    
#     scheduler = SquareRootScheduler(lr=config['lr'])

#     # If the user specified a model to preload before training, load it
#     initial_epoch = 0
#     global_step = 0
#     if config['preload']:
#         model_filename = latest_weights_file_path(config)
#         print(f'Preloading model {model_filename}')
#         state = torch.load(model_filename)
#         model.load_state_dict(state['model_state_dict'])
#         initial_epoch = state['epoch'] + 1
#         optimizer.load_state_dict(state['optimizer_state_dict'])
#         global_step = state['global_step']
#         del state

#     loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)

#     # define our custom x axis metric
#     wandb.define_metric("global_step")
#     # define which metrics will be plotted against it
#     wandb.define_metric("validation/*", step_metric="global_step")
#     wandb.define_metric("train/*", step_metric="global_step")

#     for epoch in range(initial_epoch, initial_epoch+config['num_epochs']):
#         torch.cuda.empty_cache()
# #         model.train()
#         batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
#         for batch in batch_iterator:
            
#             # moved here
#             model.train()
#             optimizer.zero_grad()

#             input = batch['input'].to(device) # (b, seq_len, d_model)

#             # Run the tensors through the encoder, decoder and the projection layer
#             encoder_output = model.encode(input) # (B, seq_len, d_model)
#             proj_output = model.project(encoder_output) # (B, num_of_labels)

#             # Compare the output with the label
#             label = batch['label'].to(device) # (B, num_of_labels)

#             # Compute the loss using a simple cross entropy
#             loss = loss_fn(proj_output, label)
#             batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

#             # Log the loss
#             wandb.log({'train/loss': loss.item(), 'global_step': global_step})

#             # Backpropagate the loss
#             loss.backward()

#             # Update the weights
#             optimizer.step()
# #             optimizer.zero_grad(set_to_none=True)

#         global_step += 1
        
#         # updating scheduler
#         if scheduler:
#             if scheduler.__module__ == lr_scheduler.__name__:
#                 # Using PyTorch In-Built scheduler
#                 scheduler.step()
#             else:
#                 # Using custom defined scheduler
#                 for param_group in optimizer.param_groups:
#                     param_group['lr'] = scheduler(epoch)

#         # Run validation at the end of every epochnbv
#         run_validation(model, val_dataloader, config, device, global_step)

#         # Save the model at the end of every epoch
#         model_filename = get_weights_file_path(config, f"{epoch:02d}")
#         torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'global_step': global_step
#         }, model_filename)

In [25]:
def run_validation_ensemble(models, validation_ds, config, device, global_step):
    # Ensure the models are in evaluation mode
    for model in models:
        model.eval()

    true_classes = []
    pred_classes = []

    with torch.no_grad():
        i=0
        for batch in validation_ds:
            inp = batch["input"].to(device)  # (b, d_model, seq_len)
            inp_mask = None

            # Check that the batch size is 1
            assert inp.size(0) == 1, "Batch size must be 1 for validation"

            # Aggregate the outputs from all models
            outputs = torch.zeros((config['num_of_labels'],)).to(device)
            model_predictions = []
            for model in models:
                out = model.encode(inp)
                out = model.project(out).squeeze(0)
                model_predictions.append(torch.argmax(out).item())
            
            
            if i==0:
                ot = torch.tensor(model_predictions)
                print(ot.shape)
                i+=1
            # Average the outputs
#             outputs /= len(models)
            #getting majority votes
            outputs = torch.tensor(model_predictions).mode().values.item()

            # Store predictions and true labels
#             pred_classes.append(torch.argmax(outputs).item())
            pred_classes.append(outputs)
            true_classes.append(torch.argmax(batch["label"].squeeze(0)).item())

        true_classes = torch.tensor(true_classes)
        pred_classes = torch.tensor(pred_classes)

    # Compute evaluation metrics
    confusion_matrix = torchmetrics.ConfusionMatrix(num_classes=config["num_of_labels"], task="multiclass")
    f1_score = torchmetrics.classification.MulticlassF1Score(num_classes=config["num_of_labels"])
    accuracy = torchmetrics.classification.MulticlassAccuracy(num_classes=config["num_of_labels"])
    recall = torchmetrics.classification.MulticlassRecall(num_classes=config["num_of_labels"])
    precision = torchmetrics.classification.MulticlassPrecision(num_classes=config["num_of_labels"])

    # Log metrics to WandB
    wandb.log({
        "validation/confusion_matrix": confusion_matrix(pred_classes, true_classes),
        "validation/accuracy": accuracy(pred_classes, true_classes), 
        "validation/recall": recall(pred_classes, true_classes), 
        "validation/precision": precision(pred_classes, true_classes), 
        "validation/f1_score": f1_score(pred_classes, true_classes),
        'global_step': global_step
    })


def get_audio_location_list(df_location):
    data = []
    ref = pd.read_csv(df_location)
    for index, row in ref.iterrows():
        data.append([row['path'], row['labels']])

    return data

def get_ds(config):
    # It only has the train split, so we divide it overselves
    ds_raw = get_audio_location_list(config['df_location'])
    
    # Keep 90% for training, 10% for validation
    train_ds_raw_size = int(0.9 * len(ds_raw))
    val_ds_raw_size = len(ds_raw) - train_ds_raw_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_raw_size, val_ds_raw_size])

#     train_ds = AudioDataset(train_ds_raw, config["seq_len"], config["d_model"])
#     val_ds = AudioDataset(val_ds_raw, config["seq_len"], config["d_model"])
    train_ds = AudioDataset(train_ds_raw, config["seq_len"], config["d_model"],augment=True)
    val_ds = AudioDataset(val_ds_raw, config["seq_len"], config["d_model"], augment=False)
    
#     print(train_ds_raw_size)
#     print(len(train_ds))

#     train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
#     val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)
    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True, num_workers=4, pin_memory=True)
    
    val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)

    return train_dataloader, val_dataloader

# def get_model(config):
#     model = build_transformer( config["seq_len"], config['num_of_labels'], d_model=config['d_model'])
#     return model
def get_models(config,num_models=4):
    models=[]
    for _ in range(num_models):
        model = build_transformer(config["seq_len"], config['num_of_labels'], d_model=config['d_model'])
        models.append(model)
        
    return models

def train_model(config, num_models=4):
    # Define the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    # Make sure the weights folder exists
    Path(config['model_folder']).mkdir(parents=True, exist_ok=True)

    train_dataloader, val_dataloader = get_ds(config)
#     model = get_model(config).to(device)
    models = get_models(config,num_models)
    models = [model.to(device) for model in models]
    
#     optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'])
    optimizers = [torch.optim.SGD(model.parameters(), lr=config['lr']) for model in models]
    schedulers = [SquareRootScheduler(lr=config['lr']) for _ in range(num_models)]
#     scheduler = SquareRootScheduler(lr=config['lr'])

    # If the user specified a model to preload before training, load it
    initial_epoch = 0
    global_step = 0
    if config['preload']:
        model_filename = latest_weights_file_path(config)
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        model.load_state_dict(state['model_state_dict'])
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']
        del state

    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)

    # define our custom x axis metric
    wandb.define_metric("global_step")
    # define which metrics will be plotted against it
    wandb.define_metric("validation/*", step_metric="global_step")
    wandb.define_metric("train/*", step_metric="global_step")

    for epoch in range(initial_epoch, initial_epoch+config['num_epochs']):
        torch.cuda.empty_cache()
        for model in models:
            model.train()
            
#         model.train()
        batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
        for batch in batch_iterator:
            for model, optimizer in zip(models, optimizers):
                
                # moved here
                optimizer.zero_grad()

                input = batch['input'].to(device) # (b, seq_len, d_model)

                # Run the tensors through the encoder, decoder and the projection layer
                encoder_output = model.encode(input) # (B, seq_len, d_model)
                proj_output = model.project(encoder_output) # (B, num_of_labels)

                # Compare the output with the label
                label = batch['label'].to(device) # (B, num_of_labels)

                # Compute the loss using a simple cross entropy
                loss = loss_fn(proj_output, label)
                batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

                # Log the loss
                wandb.log({'train/loss': loss.item(), 'global_step': global_step})

                # Backpropagate the loss
                loss.backward()

                # Update the weights
                optimizer.step()
    #             optimizer.zero_grad(set_to_none=True)

        global_step += 1
        
        # updating scheduler
        for scheduler, optimizer in zip(schedulers, optimizers):
            if scheduler:
                if scheduler.__module__ == lr_scheduler.__name__:
                    # Using PyTorch In-Built scheduler
                    scheduler.step()
                else:
                    # Using custom defined scheduler
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = scheduler(epoch)

        # Run validation at the end of every epochnbv
        run_validation_ensemble(models, val_dataloader, config, device, global_step)
        
        # Save the models at the end of every epoch
        for i, model in enumerate(models):
            model_filename = get_weights_file_path(config, f"{epoch:02d}_model_{i}")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizers[i].state_dict(),
                'global_step': global_step
            }, model_filename)

In [14]:
# #I have added this to check input data
# config = get_config()
# def get_audio_location_list(df_location):
#     data = []
#     ref = pd.read_csv(df_location)
#     for index, row in ref.iterrows():
#         data.append([row['path'], row['labels']])

#     return data

# def get_ds(config):
#     # It only has the train split, so we divide it overselves
#     ds_raw = get_audio_location_list(config['df_location'])
    
#     # Keep 90% for training, 10% for validation
#     train_ds_raw_size = int(0.9 * len(ds_raw))
#     val_ds_raw_size = len(ds_raw) - train_ds_raw_size
#     train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_raw_size, val_ds_raw_size])

# #     train_ds = AudioDataset(train_ds_raw, config["seq_len"], config["d_model"])
# #     val_ds = AudioDataset(val_ds_raw, config["seq_len"], config["d_model"])
#     train_ds = AudioDataset(train_ds_raw, config["seq_len"], config["d_model"],augment=True)
#     val_ds = AudioDataset(val_ds_raw, config["seq_len"], config["d_model"], augment=False)
    
#     print(train_ds_raw_size)
#     print(len(train_ds))

# #     train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
# #     val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)
#     train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True, num_workers=4, pin_memory=True)
    
#     val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)

#     return train_dataloader, val_dataloader



# # Make sure the weights folder exists
# Path(config['model_folder']).mkdir(parents=True, exist_ok=True)
# train_dataloader, val_dataloader = get_ds(config)
# initial_epoch = 0
# global_step = 0
# for epoch in range(initial_epoch, initial_epoch+config['num_epochs']):
#         torch.cuda.empty_cache()
#         batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
#         for batch in batch_iterator:
#             input = batch['input'] # (b, seq_len, d_model)
# #             print(input.shape) #torch.Size([16, 600, 39])

In [15]:
os.mkdir("Global_dataset_ravdess_tess_weights")                         

In [ ]:
# !pip install wandb
# !wandb login
# WANDB_API_KEY = "5917e81ca177aa743d512c104ea62b82dc6da9f5"


In [ ]:
# wandb login --relogin                 v          x                                                                                                                                      v                                                                                                                v                        v                                              v                                                                  b v           v                                                                             0                     x                   x                                                                z                                                  x         